In [1]:
import cv2
import numpy as np
import time

#start reading webcam
cap = cv2.VideoCapture(0)

#read initial frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

#get screen dimensions
frame_height, frame_width = frame1.shape[:2]

#define center tolerance
center_tolerance = 50

#variable to track if an object is already centered and a photo has been taken
object_in_center = False

#initialize contour_feed to avoid reference before assignment
contour_feed = np.zeros_like(frame1)

#initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2()

#motion detection happens here
while cap.isOpened():
    #compute the absolute difference between frames
    diff = cv2.absdiff(frame1, frame2)
    
    #convert the difference to grayscale
    gray_motion = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    #apply gaussian blur
    blur_motion = cv2.GaussianBlur(gray_motion, (5, 5), 0)
    
    #get threshold
    _, thresh_motion = cv2.threshold(blur_motion, 20, 255, cv2.THRESH_BINARY)
    
    #dilate threshold
    dilated_motion = cv2.dilate(thresh_motion, None, iterations=3)
    
    #find contours
    contours_motion, _ = cv2.findContours(dilated_motion, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    object_centered_now = False
    
    #draw contours around detected motion
    for contour in contours_motion:
        if cv2.contourArea(contour) < 700:
            continue
        
        #get bounding box coordinates for each contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #draw rectangle around detected object
        cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        #calculate center of bounding box
        center_x = x + w // 2
        center_y = y + h // 2
        
        #check if the object is approximately in the center of the screen
        screen_center_x = frame_width // 2
        screen_center_y = frame_height // 2
        
        if (abs(center_x - screen_center_x) <= center_tolerance and
            abs(center_y - screen_center_y) <= center_tolerance):
            object_centered_now = True

            #only take a screenshot if the object has not been centered before
            if not object_in_center:
                screenshot_filename = f'screenshot_{int(time.time())}.png'
                cv2.imwrite(screenshot_filename, contour_feed) 


KeyboardInterrupt: 